In [9]:
from dotenv import load_dotenv
load_dotenv() # this will set the Google API key as an environment variable from .env file

from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages 
from typing import Annotated

In [2]:
from langchain.chat_models import init_chat_model
# using langchain to create an application that uses a conversational LLM

In [10]:
# create a llm by calling this chat model
llm = init_chat_model('google_genai:gemini-2.0-flash')

# llm.invoke('who was the first person to walk the moon?')

class State(TypedDict):
    # messages: list #exchanging messages so a list to store the history, but this doesnt give the conversation format
    messages: Annotated[list, add_messages] # this will preserve the original message, add_messages is builtin in langgraph
    

def chatbot(state: State) -> State:
    return {'messages': [llm.invoke(state['messages'])]}

builder = StateGraph(State)
builder.add_node('chatbot_node', chatbot)

builder.add_edge(START, 'chatbot_node')
builder.add_edge('chatbot_node', END)

graph = builder.compile()

E0000 00:00:1759706275.862611  335594 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [11]:
message = {'role':'user', 'content':'Who walked on the moon for the first time? print only the name'}
response = graph.invoke({'messages':[message]})

response['messages']

[HumanMessage(content='Who walked on the moon for the first time? print only the name', additional_kwargs={}, response_metadata={}, id='11380e84-bae4-4575-8359-21821cba021f'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a8ca4946-7563-4c01-8838-372baeeef2a2-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [12]:
# This method of storing messages in a list works but we might run into issues. Langgraph offers a build-in method to have memory
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit", "exit"}:
        break
    if state is None:
        state: State = { 
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state ["messages"].append({"role": "user", "content": in_message})
        
state = graph.invoke(state)
print("Bot:", state ["messages"] [-1].content)

/Users/varadthikekar/PycharmProjects/LangGraph learning/.venv/lib/python3.12/site-packages/langchain_google_genai/chat_models.py:2007: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


Bot: The first man to walk on the Moon was **Neil Armstrong**.

The year he walked on the Moon was **1969**.
